# 4. Gt, Lt, Between, Within, Mean, Dedup, Where, Neq, Limit, Range, Skip, Tail, TimeLimit
#### By Kwan Yin Andrew Chau

In [1]:
import nbfinder
from loader import g
from loader import P
from loader import __
from loader import hkVertexId
from loader import sydneyVertexId

## gt(), lt()
__Get destinations from HK that have more than 5 runways__

In [2]:
g.V(hkVertexId).out('route').has('runways', P.gt(5)).valueMap('desc', 'runways').fold().next()

[{'runways': [6], 'desc': ['Boston Logan']},
 {'runways': [7], 'desc': ['Dallas/Fort Worth International Airport']},
 {'runways': [8], 'desc': ["Chicago O'Hare International Airport"]},
 {'runways': [6], 'desc': ['Amsterdam Airport Schiphol']}]

## between()
__Get destinations from HK that have 6 to 8 runways__

In [3]:
g.V(hkVertexId).out('route').has('runways', P.between(6, 8)).valueMap('desc', 'runways').fold().next()

[{'runways': [6], 'desc': ['Boston Logan']},
 {'runways': [7], 'desc': ['Dallas/Fort Worth International Airport']},
 {'runways': [6], 'desc': ['Amsterdam Airport Schiphol']}]

## within()
__Count of vertices that are within region "AU-NSW" and "AU-VIC"__

In [4]:
g.V().has('region', P.within('AU-NSW', 'AU-VIC')).values('desc').fold().next()

['Orange Airport',
 'Moruya Airport',
 'Merimbula Airport',
 'Tamworth Airport',
 'Melbourne Essendon Airport',
 'Moree Airport',
 'Ballina Byron Gateway Airport',
 'Wagga Wagga City Airport',
 'Sydney Kingsford Smith',
 'Taree Airport',
 'Newcastle Airport',
 'Dubbo City Regional Airport',
 'Warrnambool Airport',
 'Parkes Airport',
 'Grafton Airport',
 'Armidale Airport',
 'Bathurst Airport',
 'Mildura Airport',
 'Narrabri Airport',
 'Broken Hill Airport',
 'Sydney Bankstown Airport',
 'Coffs Harbour Airport',
 'Melbourne International Airport',
 'Avalon Airport',
 'Port Macquarie Airport',
 'Griffith Airport',
 'Lord Howe Island Airport',
 'Narrandera Airport',
 'Portland Airport',
 'Albury Airport',
 'Lismore Airport',
 'Gold Coast Airport']

## mean()

__Get the mean between the number of runways of airports within regions "AU-NSW" and "AU-VIC"__

In [5]:
g.V().has('region', P.within('AU-NSW', 'AU-VIC')).values('runways').mean().next()

1.875

## dedup()

__Get airports that have destinations the HK airport does, and remove duplicates__

In [6]:
x1 = g.V(hkVertexId).out('route').in_('route').dedup().count().next()
g.V(hkVertexId).out('route').in_('route').values('desc').dedup().fold().next()

['Fort Lauderdale/Hollywood International Airport',
 'Ronald Reagan Washington National Airport',
 'Hartsfield - Jackson Atlanta International Airport',
 'Palm Beach International Airport',
 'Dallas/Fort Worth International Airport',
 'Baltimore/Washington International Airport',
 'Raleigh-Durham',
 'Orlando International Airport',
 'Seattle-Tacoma',
 'Austin Bergstrom International Airport',
 'San Francisco International Airport',
 'Tampa International Airport',
 'Washington Dulles International Airport',
 'Norman Y. Mineta San Jose International Airport',
 'Houston Hobby',
 'Las Vegas Mc Carran',
 'George Bush Intercontinental',
 'Cleveland, Hopkins International Airport',
 'Vancouver',
 'New Orleans L. Armstrong',
 'New York John F. Kennedy International Airport',
 'Paris Charles de Gaulle',
 'London Heathrow',
 'Miami International Airport',
 'Tokyo Narita',
 'London Gatwick',
 "Chicago O'Hare International Airport",
 'Dublin International Airport',
 'San Diego Lindbergh',
 'Beijin

__Remove duplicates by number of runways__

In [7]:
g.V().has('region','AU-NSW').dedup().by('runways').values('code','runways').fold().next()

['OAG', 2, 'MIM', 1, 'TMW', 4, 'SYD', 3]

__Remove duplicates where there is the same A and C even if B is different__

In [8]:
g.V(hkVertexId).as_('a').V(sydneyVertexId).as_('c').both().as_('b').limit(10).dedup('a','c').select('a','b','c').fold().next()

[{'a': v[98344], 'b': v[8328], 'c': v[90152]}]

## where(), neq()

Exclude the HK airport itself

In [9]:
x2 = g.V(hkVertexId).as_('hk').out('route').in_('route').where(P.neq('hk')).dedup().count().next()
assert x1 == x2 + 1
x2

1879

## limit()
Limit the amount of results to "n"

In [10]:
n = 5
count = g.V(hkVertexId).out('route').limit(n).count().next()
assert n == count
x1 = g.V(hkVertexId).out('route').limit(n).fold().next()
x1

[v[4296], v[8328], v[12488], v[16440], v[24712]]

## range()
Equivalent to using a range starting from 0

In [11]:
x2 = g.V(hkVertexId).out('route').range(0, n).fold().next()
x2

[v[4296], v[8328], v[12488], v[16440], v[24712]]

__Get the results from "n" to the end__

In [12]:
# x1 = g.V(hkVertexId).out('route').range(n, -1).fold().next()
# x1

## skip()
Equivalent to skipping by "n"

In [13]:
# x2 = g.V(hkVertexId).out('route').skip(n).fold().next()
# assert x1 == x2

## tail()
__Get the last "n" results__

In [14]:
g.V(hkVertexId).out('route').tail(n).fold().next()

[v[41558224], v[41566416], v[41578704], v[41590872], v[41594968]]

There is no guarantee to the order of ranges, it is often based on order put into graph

## timeLimit()
Limit the query to however much can be processed within 10 ms

In [15]:
g.V(hkVertexId).repeat(__.timeLimit(10).out()).until(__.has('code','SYD')).path().by('code').next()

['HKG', 'SYD']